#### TODO
* Operation `::`, `++`
  * `op2.::(op1)`
  * `op1.++(op2)`
  * operator has its precedence.
* variable length argument
* hasElement -> assign over inheritance

In [1]:
List(1,2,3,4)
1::2::3::4::Nil

val x = 1::2::3::Nil
val y = 4::5::Nil
x ++ y

res0_0: List[Int] = List(1, 2, 3, 4)
res0_1: List[Int] = List(1, 2, 3, 4)
x: List[Int] = List(1, 2, 3)
y: List[Int] = List(4, 5)
res0_4: List[Int] = List(1, 2, 3, 4, 5)

### List Syntax
* `List(elements)`: list constructor
* `++`: list append, same as `.++()`
* `::`: value append, same as `.::()`

In [2]:
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
sealed abstract class MyList[A] extends Iter[A] {
    def ::(hd: A): MyList[A] = MyCons(hd, this)
    def ++(lst: MyList[A]): MyList[A]
}
case class MyNil[A]() extends MyList[A] {
    def getValue = None
    def getNext = throw new Exception("...")
    def ++(lst: MyList[A]) = lst
}
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A] {
    def getValue = Some(hd)
    def getNext = tl
    def ++(lst: MyList[A]) = MyCons(hd, tl++lst)
}
object MyList {
    def ::[A](v1: A, v2: A): MyList[A] = MyCons(v1, MyCons(v2, MyNil()))
    def ::[A](v: A, lst: MyList[A]): MyList[A] = MyCons(v, lst)
    def ++[A](lst1: MyList[A], lst2: MyList[A]): MyList[A] = lst1++lst2
}
val x = MyCons(2, MyCons(3, MyNil()))
val y = MyCons(2, MyCons(3, MyNil()))
MyList.::(1, 2)
MyList.::(1, MyNil[Int]())
MyList.++(x, y)

// 'op2.::(op1)'
// 'op1.++(op2)'
MyNil().::(1)
1::MyNil()

defined class Iter
defined class MyList
defined class MyNil
defined class MyCons
defined object MyList
x: MyCons[Int] = MyCons(2, MyCons(3, MyNil()))
y: MyCons[Int] = MyCons(2, MyCons(3, MyNil()))
res1_7: MyList[Int] = MyCons(1, MyCons(2, MyNil()))
res1_8: MyList[Int] = MyCons(1, MyNil())
res1_9: MyList[Int] = MyCons(2, MyCons(3, MyCons(2, MyCons(3, MyNil()))))
res1_10: MyList[Int] = MyCons(1, MyNil())
res1_11: MyList[Int] = MyCons(1, MyNil())

In [3]:
abstract class Iterable[A] {
    type iter_t
    def iter: iter_t
    def getValue(i: iter_t): Option[A]
    def getNext(i: iter_t): iter_t
}

defined class Iterable

In [4]:
abstract class IterableHE[A] extends Iterable[A] {
    def eq(a:A, b:A) : Boolean
    def hasElement(a: A) : Boolean = {
        def hasElementIter(i: iter_t) : Boolean =
            getValue(i) match {
                case None => false
                case Some(n) =>
                    if (eq(a,n)) true
                    else hasElementIter(getNext(i))
            }
        hasElementIter(iter)
    }
}

defined class IterableHE

In [7]:
abstract class IterableHE[A] extends Iterable[A] {
    def eq(a:A, b:A) : Boolean
    def hasElement(a: A) : Boolean
}

object IterableHE {
    def hasElement[A](eq: (A,A)=>Boolean, xs: Iterable[A], a: A) : Boolean = {
        def hasElementIter(i: xs.iter_t) : Boolean =
            xs.getValue(i) match {
                case None => false
                case Some(n) =>
                    if (eq(a,n)) true
                    else hasElementIter(xs.getNext(i))
            }
        hasElementIter(xs.iter)
    }
}


defined class IterableHE
defined object IterableHE

In [9]:
sealed abstract class MyTree[A] extends IterableHE[A] {
    type iter_t = List[A]
    def getValue(i : List[A]) : Option[A] = i.headOption
    def getNext(i: List[A]) : List[A] = i.tail
    // add this line
    def hasElement(a: A) = IterableHE.hasElement(this.eq, this, a)
}
// why do we have to pass '_eq'
case class Empty[A](_eq: (A,A)=>Boolean) extends MyTree[A] {
    def eq(a: A, b: A) = _eq(a, b)
    val iter : List[A] = Nil
}
case class Node[A](_eq: (A,A)=>Boolean, value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A] {
    def eq(a: A, b: A) = _eq(a, b)
    val iter : List[A] = value :: (left.iter ++ right.iter)
}

defined class MyTree
defined class Empty
defined class Node

### Prime Number Example
* `primes`: current prime list
  * `p := new Primes(11, [2,3,5,7])`
  * `p.getNext = new Primes(13, [2,3,5,7,11])`
* `forall` List function
  * if all elements are `true`

In [37]:
// should we pass prime and primes? (danger) => Multiple constructor
class Primes(val prime: Int, val primes: List[Int]) {
    def getNext: Primes = {
        val p = computeNextPrime(prime + 2)
        new Primes(p, primes ++ (p :: Nil))
    }
    def computeNextPrime(n: Int) : Int =
        if (primes.forall((p:Int) => n%p != 0)) n
        else computeNextPrime(n+2)
}
def nthPrime(n: Int): Int = {
    def go(primes: Primes, k: Int): Int =
        if (k <= 1) primes.prime
        else go(primes.getNext, k - 1)
    if (n == 0) 2 else go(new Primes(3, List(3)), n)
}
new Primes(3, List(3)).prime
nthPrime(20000)

defined class Primes
defined function nthPrime
res36_2: Int = 3
res36_3: Int = 224743

#### to make `Primes(prime, primes)` private

In [38]:
class Primes private (val prime: Int, val primes: List[Int]) {
    def this() = this(3, List(3))
    def getValue: Int = prime
    def getNext: Primes = {
        val p = computeNextPrime(prime + 2)
        new Primes(p, primes ++ (p :: Nil))
    }
    private def computeNextPrime(n: Int) : Int =
        if (primes.forall((p:Int) => n%p != 0)) n
        else computeNextPrime(n+2)
}
def nthPrime(n: Int): Int = {
    def go(primes: Primes, k: Int): Int =
        if (k <= 1) primes.prime
        else go(primes.getNext, k - 1)
    if (n == 0) 2 else go(new Primes, n)
}
new Primes().getValue
// not allowed
// new Primes(3, List(3)).getValue
nthPrime(20000)

defined class Primes
defined function nthPrime
res37_2: Int = 3
res37_3: Int = 224743

### OOP vs Type Class
* OOP has separated implementation of elimination & constructor while Type Class doesn't.

In [39]:
abstract class PrimesSig[P] {
    def makeNew : P
    def prime(p: P) : Int
    def getNext(p: P) : P
}
def nthPrime[P](tc: PrimesSig[P])(n: Int): Int = {
    def go(p: P, k: Int): Int =
        if (k <= 1) tc.prime(p) else go(tc.getNext(p), k - 1)
        if (n == 0) 2 else go(tc.makeNew, n)
}
class Primes private (val prime: Int, protected val primes: List[Int]) {
    def this() = this(3, List(3))
    def getNext: Primes = {
        val p = computeNextPrime(prime + 2)
        new Primes(p, primes ++ (p :: Nil))
    }
    private def computeNextPrime(n: Int) : Int =
        if (primes.forall((p:Int) => n%p != 0)) n
        else computeNextPrime(n+2)
}
class PrimesImpl extends PrimesSig[Primes] {
    def makeNew = new Primes
    def prime(p: Primes) = p.prime
    def getNext(p: Primes) = p.getNext
}
val primesTC = new PrimesImpl
nthPrime(primesTC)(10000)

defined class PrimesSig
defined function nthPrime
defined class Primes
defined class PrimesImpl
primesTC: PrimesImpl = ammonite.$sess.cmd38$Helper$PrimesImpl@3304ce27
res38_5: Int = 104743

In [43]:
abstract class PrimesSig[P] {
    def makeNew : P
    def prime(p: P) : Int
    def getNext(p: P) : P
}
def nthPrime[P](n: Int)(implicit tc: PrimesSig[P]): Int = {
    def go(p: P, k: Int): Int =
        if (k <= 1) tc.prime(p) else go(tc.getNext(p), k - 1)
        if (n == 0) 2 else go(tc.makeNew, n)
}
class Primes private (val prime: Int, protected val primes: List[Int]) {
    def this() = this(3, List(3))
    def getNext: Primes = {
        val p = computeNextPrime(prime + 2)
        new Primes(p, primes ++ (p :: Nil))
    }
    private def computeNextPrime(n: Int) : Int =
        if (primes.forall((p:Int) => n%p != 0)) n
        else computeNextPrime(n+2)
}
class PrimesImpl extends PrimesSig[Primes] {
    def makeNew = new Primes
    def prime(p: Primes) = p.prime
    def getNext(p: Primes) = p.getNext
}
implicit val primesTC = new PrimesImpl
nthPrime(10000)

defined class PrimesSig
defined function nthPrime
defined class Primes
defined class PrimesImpl
primesTC: PrimesImpl = ammonite.$sess.cmd42$Helper$PrimesImpl@7cac080f
res42_5: Int = 104743